In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from Carabins.data_extraction.analysis import get_subject_metrics, get_traits_rapides_params

In [7]:

dbname = 'data_carabins.db'
full_data = pd.merge(get_subject_metrics(dbname), get_traits_rapides_params(dbname), left_on='id', right_on='subject_id')

# print all available data columns
print(full_data.columns.values)
selected_data = full_data[['t0', 'D1', 'mu1', 'ss1', 'D2', 'mu2', 'ss2', 'SNR', 'pain']]
# interpret no answer as 0 pain
selected_data['pain'] = selected_data['pain'].fillna(0)
selected_data = selected_data.dropna()
X = selected_data[['t0', 'D1', 'mu1', 'ss1', 'D2', 'mu2', 'ss2', 'SNR']].values
y = selected_data[['pain']].values

['id' 'position' 'status' 'height' 'weight' 'fat' 'arm_length'
 'asym_drop_box' 'asym_tuck_jump' 'hop_g1' 'hop_g2' 'hop_d1' 'hop_d2'
 'hop3_g1' 'hop3_g2' 'hop3_d1' 'hop3_d2' 'hop3_cr_g1' 'hop3_cr_g2'
 'hop3_cr_d1' 'hop3_cr_d2' 're_gh_g1' 're_gh_g2' 're_gh_d1' 're_gh_d2'
 'flex_g1' 'flex_g2' 'flex_d1' 'flex_d2' 'scap_g1' 'scap_g2' 'scap_d1'
 'scap_d2' 'date' 'injury' 'pain' 'gen' 'phys' 'men' 'act' 'mot'
 'avg_fatigue' 'subject_id' 't0' 'D1' 'mu1' 'ss1' 'D2' 'mu2' 'ss2' 'SNR'
 'post_exercise']


C:\Users\ettre\AppData\Local\Continuum\miniconda3\envs\Sebastien\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
t_data, v_data, t_labels, v_labels = train_test_split(X, y, test_size=0.2)
t_data.shape

(72, 8)

In [9]:
model = keras.Sequential([
    keras.layers.Dense(6,activation=tf.nn.tanh),
    keras.layers.Dense(12, activation=tf.nn.tanh),
    keras.layers.Dense(24, activation=tf.nn.tanh),
    keras.layers.Dense(6, activation=tf.nn.softmax)
])
model.compile(optimizer=keras.optimizers.Nadam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [37]:
model.fit(t_data, t_labels, epochs=10)

Epoch 1/10
60/60 [==============================] - 2s 31ms/sample - loss: 1.9961 - acc: 0.0667
Epoch 2/10
60/60 [==============================] - 0s 501us/sample - loss: 1.9026 - acc: 0.0667
Epoch 3/10
60/60 [==============================] - 0s 150us/sample - loss: 1.8205 - acc: 0.0667
Epoch 4/10
60/60 [==============================] - 0s 117us/sample - loss: 1.7400 - acc: 0.0667
Epoch 5/10
60/60 [==============================] - 0s 117us/sample - loss: 1.6577 - acc: 0.0667
Epoch 6/10
60/60 [==============================] - 0s 150us/sample - loss: 1.5734 - acc: 0.3833
Epoch 7/10
60/60 [==============================] - 0s 150us/sample - loss: 1.4849 - acc: 0.7833
Epoch 8/10
60/60 [==============================] - 0s 117us/sample - loss: 1.4003 - acc: 0.7833
Epoch 9/10
60/60 [==============================] - 0s 184us/sample - loss: 1.3153 - acc: 0.7833
Epoch 10/10
60/60 [==============================] - 0s 133us/sample - loss: 1.2315 - acc: 0.7833


In [38]:
test_loss, test_acc = model.evaluate(v_data, v_labels)

15/15 [==============================] - 0s 9ms/sample - loss: 1.1680 - acc: 0.8000


In [39]:
test_loss

1.1679993867874146

In [40]:
test_acc

0.8

In [50]:
predictions = model.predict(v_data)
predictions.shape
print(np.argmax(predictions, axis=1))
print(np.transpose(v_labels).astype(int)[0])

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[4 0 0 0 0 0 0 0 0 0 0 0 0 2 0]


In [ ]:
print(v_labels)